In [4]:
import boto3
from sagemaker import Session, get_execution_role
from sagemaker.xgboost.model import XGBoostModel

# Initialize SageMaker session and get execution role
session = Session()
role = get_execution_role()  # If in SageMaker Notebook; otherwise hardcode your role ARN

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [5]:
import tarfile

with tarfile.open('xgb-model.tar.gz', 'w:gz') as tar:
    tar.add('xgb-model.deprecated', arcname='xgboost-model')  # Rename inside archive
s3 = boto3.client('s3')

bucket_name = 'kaggle-housing-pipeline-data'
model_key = 'models/xgb-model.tar.gz'

# Upload model artifact if not already done
s3.upload_file('xgb-model.tar.gz', bucket_name, model_key)

model_data = f's3://{bucket_name}/{model_key}'

In [6]:
model = XGBoostModel(
    model_data=model_data,
    role=role,
    framework_version='1.5-1',  # Make sure this matches a supported SageMaker XGBoost version
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large'
)

print(f"Endpoint name: {predictor.endpoint_name}")


------!Endpoint name: sagemaker-xgboost-2025-07-22-02-26-04-456


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:14                                                                                   │
│                                                                                                  │
│   11                                                                                             │
│   12 print(f"Endpoint name: {predictor.endpoint_name}")                                          │
│   13                                                                                             │
│ ❱ 14 with open('models/endpoint_name.txt', 'w') as f:                                            │
│   15 │   f.write(predictor.endpoint_name)                                                        │
│   16                                                                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/IPython/core/interactiveshell.py:324 in _modified_open   │
│                                                                                                  │
│    321 │   │   │   "you can use builtins' open."                                                 │
│    322 │   │   )                                                                                 │
│    323 │                                                                                         │
│ ❱  324 │   return io_open(file, *args, **kwargs)                                                 │
│    325                                                                                           │
│    326 class InteractiveShell(SingletonConfigurable):                                            │
│    327 │   """An enhanced, interactive shell for Python."""                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: [Errno 2] No such file or directory: 'models/endpoint_name.txt'

In [9]:

with open('model/endpoint_name.txt', 'w') as f:
    f.write(predictor.endpoint_name)